In [1]:
import jupyter_black
from IPython.display import display


jupyter_black.load(line_length=999)

In [2]:
import os
import pandas as pd
import torch
import torchvision
from tqdm.notebook import tqdm

In [3]:
scores = pd.read_excel("量表最终版.xlsx")
scores = scores[["提交答卷时间", "账号", "PHQ-9总分", "GAD-7总分", "AIS总分"]]

scores["提交答卷时间"] = pd.to_datetime(scores["提交答卷时间"], format="%Y/%m/%d %H:%M:%S")
scores["提交答卷时间"] = scores["提交答卷时间"].dt.date

scores = scores[(scores["账号"] == "gzj") | (scores["账号"] == "hm")]
scores.rename(
    columns={
        "提交答卷时间": "Date",
        "账号": "SID",
        "PHQ-9总分": "PHQ-9",
        "GAD-7总分": "GAD-7",
        "AIS总分": "AIS",
    },
    inplace=True,
)
scores.sort_values(by="Date", inplace=True)
scores

,Date,SID,PHQ-9,GAD-7,AIS
30,2024-10-22,hm,8,6,3
26,2024-10-23,gzj,4,3,5
28,2024-11-06,gzj,3,1,8
33,2024-11-06,hm,7,5,5
27,2024-11-13,gzj,4,0,5
31,2024-11-13,hm,7,6,6
32,2024-11-29,hm,7,6,6
29,2024-12-02,gzj,5,6,6
34,2024-12-02,hm,7,6,1


In [4]:
def interpolate_scores(df: pd.DataFrame) -> pd.DataFrame:
    df["Date"] = pd.to_datetime(df["Date"])
    interpolated_dfs = []
    for sid, group in df.groupby("SID"):
        group = group.set_index("Date")

        for col in ["PHQ-9", "GAD-7", "AIS"]:
            group[col] = pd.to_numeric(group[col], errors="coerce")

        group = group.resample("D").interpolate(method="time")
        group["SID"] = sid
        interpolated_dfs.append(group)

    final_df = pd.concat(interpolated_dfs)
    final_df = final_df.reset_index()
    return final_df


scores = interpolate_scores(scores.copy())
scores

C:\Users\von\AppData\Local\Temp\ipykernel_12556\633350542.py:10: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.resample("D").interpolate(method="time")
C:\Users\von\AppData\Local\Temp\ipykernel_12556\633350542.py:10: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.resample("D").interpolate(method="time")


,Date,SID,PHQ-9,GAD-7,AIS
0,2024-10-23,gzj,4.000000,3.000000,5.000000
1,2024-10-24,gzj,3.928571,2.857143,5.214286
2,2024-10-25,gzj,3.857143,2.714286,5.428571
3,2024-10-26,gzj,3.785714,2.571429,5.642857
4,2024-10-27,gzj,3.714286,2.428571,5.857143
...,...,...,...,...,...
78,2024-11-28,hm,7.000000,6.000000,6.000000
79,2024-11-29,hm,7.000000,6.000000,6.000000
80,2024-11-30,hm,7.000000,6.000000,4.333333
81,2024-12-01,hm,7.000000,6.000000,2.666667


In [ ]:
def add_scores(dump_dir: str, scores_df: pd.DataFrame, output_dir: str = "data_add"):
    scores_df["Date"] = pd.to_datetime(scores_df["Date"]).dt.strftime("%Y-%m-%d")
    os.makedirs(output_dir, exist_ok=True)

    for feature_dir in tqdm(os.listdir(dump_dir), desc="Processing features"):
        feature_path = os.path.join(dump_dir, feature_dir)
        if os.path.isdir(feature_path):
            output_feature_path = os.path.join(output_dir, feature_dir)
            os.makedirs(output_feature_path, exist_ok=True)

            for filename in os.listdir(feature_path):
                if filename.endswith(".csv"):
                    file_path = os.path.join(feature_path, filename)
                    date_str, _ = os.path.splitext(filename)
                    date_str = pd.to_datetime(date_str).strftime("%Y-%m-%d")

                    df = pd.read_csv(file_path, dtype=str)

                    sid = df["SID"].iloc[0]

                    match = scores_df[(scores_df["SID"] == sid) & (scores_df["Date"] == date_str)]

                    if not match.empty:
                        phq9 = round(match["PHQ-9"].iloc[0], 4)
                        gad7 = round(match["GAD-7"].iloc[0], 4)
                        ais = round(match["AIS"].iloc[0], 4)

                        df["PHQ-9"] = phq9
                        df["GAD-7"] = gad7
                        df["AIS"] = ais

                        output_file_path = os.path.join(output_feature_path, filename)
                        df.to_csv(output_file_path, index=False)

In [6]:
dump_dir = "data_csv"
add_scores(dump_dir, scores)

Processing features:   0%|          | 0/9 [00:00<?, ?it/s]